In [14]:
import pandas as pd
import gcsfs
fs = gcsfs.GCSFileSystem()

path = f'gs://amazon-home-and-kitchen/full_train_data_txt_processed.csv'
df = pd.read_csv(path, dtype={'price': float,'subtitle': str }, na_values=['—'], nrows=100000)

df.head()


,main_category,title_x,average_rating,rating_number,features,description,price,images_x,videos,store,...,rating,title_y,text,images_y,asin,user_id,timestamp,helpful_vote,verified_purchase,processed_text
0,Amazon Home,jersey seating 2 x Vinyl Air Lift Adjustable S...,4.3,167,"['Sleek chrome metal base, seat covered in Red...",[],NaN,[{'thumb': 'https://m.media-amazon.com/images/...,"[{'title': 'Small and Stylish Barstools ', 'ur...",jersey seating®,...,5,Five Stars,nice,[],B00KKU8HVE,AHED326L5JZMCI2GAXFXNKZGQKNA,2015-07-24 14:37:07.000,0,True,nice
1,Amazon Home,jersey seating 2 x Vinyl Air Lift Adjustable S...,4.3,167,"['Sleek chrome metal base, seat covered in Red...",[],NaN,[{'thumb': 'https://m.media-amazon.com/images/...,"[{'title': 'Small and Stylish Barstools ', 'ur...",jersey seating®,...,5,Love it!,These chairs are beautiful bought 4 of these i...,[],B00KKU8HVE,AHROHCEEEGFMNYMMWXQCHLPBVVLQ,2016-11-24 17:43:29.000,0,True,chair beautiful bought 4 black color stylish c...
2,Amazon Home,jersey seating 2 x Vinyl Air Lift Adjustable S...,4.3,167,"['Sleek chrome metal base, seat covered in Red...",[],NaN,[{'thumb': 'https://m.media-amazon.com/images/...,"[{'title': 'Small and Stylish Barstools ', 'ur...",jersey seating®,...,1,Returned for bad build quality,Horible build quality. The stools don't lock i...,[],B00KKU8HUA,AHH4QDZVVTYKMCM7UYL7KD24GIPA,2017-02-11 08:46:16.000,0,True,horible build quality stool dont lock place si...
3,Amazon Home,jersey seating 2 x Vinyl Air Lift Adjustable S...,4.3,167,"['Sleek chrome metal base, seat covered in Red...",[],NaN,[{'thumb': 'https://m.media-amazon.com/images/...,"[{'title': 'Small and Stylish Barstools ', 'ur...",jersey seating®,...,5,"Great quality, especially for the price!","Great quality, especially for the price!<br />...",[],B00KKU8HVE,AHBDGI6TOGE33U5W6VEP55NEKC6Q,2016-12-14 02:22:24.000,0,True,great quality especially pricebr br ive swivel...
4,Amazon Home,jersey seating 2 x Vinyl Air Lift Adjustable S...,4.3,167,"['Sleek chrome metal base, seat covered in Red...",[],NaN,[{'thumb': 'https://m.media-amazon.com/images/...,"[{'title': 'Small and Stylish Barstools ', 'ur...",jersey seating®,...,5,Space saver,Great for the price! Worked perfectly for our ...,[],B00KKU8HVE,AHTYSHSOLY4YTIPY53BRRJ2T325Q,2018-07-02 01:49:55.194,0,True,great price worked perfectly small space


In [8]:
df.shape

(10000, 26)

In [15]:
df = df.dropna(subset=['processed_text'])

In [16]:
import pandas as pd
import gensim
from gensim import corpora
from nltk.corpus import stopwords
import nltk

# Download nltk stopwords
nltk.download('stopwords')

# Sample data structure
# df = pd.DataFrame({'product_id': [1, 1, 2, 2], 'review': ["This product is great", "Amazing quality", "Not worth the price", "Could be better"]})

# Preprocess text data
def preprocess_text(text):
    stop_words = set(stopwords.words('english'))
    tokens = gensim.utils.simple_preprocess(text, deacc=True)  # Tokenize and remove punctuation
    return [word for word in tokens if word not in stop_words]

# Apply preprocessing
df['cleaned_review'] = df['processed_text'].apply(preprocess_text)

# Create dictionary and corpus
dictionary = corpora.Dictionary(df['cleaned_review'])
corpus = [dictionary.doc2bow(text) for text in df['cleaned_review']]

# Train the LDA model
num_topics = 5  # Define the number of topics
lda_model = gensim.models.LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=10, random_state=42)

# Get the top words for each topic
topics = lda_model.print_topics(num_words=5)
topic_words = {i: [word for word, _ in lda_model.show_topic(i, topn=5)] for i in range(num_topics)}

# Function to get dominant topic
def get_dominant_topic_words(lda_model, bow):
    topics = lda_model.get_document_topics(bow)
    dominant_topic_id = sorted(topics, key=lambda x: x[1], reverse=True)[0][0]
    return topic_words[dominant_topic_id]  # Return words instead of topic ID

# Apply function to get dominant topic words for each review
df['dominant_topic_words'] = [get_dominant_topic_words(lda_model, bow) for bow in corpus]
top_topics_by_product = df.groupby('title_x')['dominant_topic_words'].apply(lambda x: x.value_counts().index[0]).reset_index()

# # Display top topics by product
# print(top_topics_by_product)

df_sorted_by_product = top_topics_by_product.explode('dominant_topic_words').reset_index(drop=True)

# Step 2: Rename columns for clarity
df_sorted_by_product.columns = ['product_id', 'topic']


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jupyter/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [21]:

from google.cloud import storage
bucket_name = 'amazon-home-and-kitchen'
destination_blob_name = '/tmp/topic_test_2024-11-09.csv'

# Save DataFrame as CSV locally first
df_sorted_by_product.to_csv('/tmp/topic_test_2024-11-09.csv', index=False)

# Initialize a GCS client and upload wwthe file
client = storage.Client()
bucket = client.bucket(bucket_name)
blob = bucket.blob(destination_blob_name)
blob.upload_from_filename('/tmp/topic_test_2024-11-09.csv')

In [23]:
df_sorted_by_product.head()

,product_id,topic
0,""" OCCASIONS "" 10 Pieces Pack Pressed Vinyl Met...",sheet
1,""" OCCASIONS "" 10 Pieces Pack Pressed Vinyl Met...",color
2,""" OCCASIONS "" 10 Pieces Pack Pressed Vinyl Met...",soft
3,""" OCCASIONS "" 10 Pieces Pack Pressed Vinyl Met...",love
4,""" OCCASIONS "" 10 Pieces Pack Pressed Vinyl Met...",nice


In [13]:
exploded_df = top_topics_by_product.explode('dominant_topic_words').reset_index(drop=True)

# Step 2: Rename columns for clarity
exploded_df.columns = ['product_id', 'topic']

print(exploded_df)

                                             product_id  topic
0     16X20X1 (15.75 x 19.75)Accumulair Platinum 1-I...  sheet
1     16X20X1 (15.75 x 19.75)Accumulair Platinum 1-I...     br
2     16X20X1 (15.75 x 19.75)Accumulair Platinum 1-I...   like
3     16X20X1 (15.75 x 19.75)Accumulair Platinum 1-I...    one
4     16X20X1 (15.75 x 19.75)Accumulair Platinum 1-I...    get
...                                                 ...    ...
1395  wall26 - Canvas Wall Art - Blue Oldtimer Farmi...    one
1396  wall26 - Canvas Wall Art - Blue Oldtimer Farmi...   back
1397  wall26 - Canvas Wall Art - Blue Oldtimer Farmi...  frame
1398  wall26 - Canvas Wall Art - Blue Oldtimer Farmi...  would
1399  wall26 - Canvas Wall Art - Blue Oldtimer Farmi...     br

[1400 rows x 2 columns]


In [6]:
exploded_df = top_topics_by_product.explode('dominant_topic_words').reset_index(drop=True)

# Step 2: Rename columns for clarity
exploded_df.columns = ['product_id', 'topic']

# Step 3: Reorder columns (optional) for easier reading
final_df = exploded_df[['topic', 'product_id']]

print(final_df)

      topic                                         product_id
0     sheet  16X20X1 (15.75 x 19.75)Accumulair Platinum 1-I...
1        br  16X20X1 (15.75 x 19.75)Accumulair Platinum 1-I...
2      like  16X20X1 (15.75 x 19.75)Accumulair Platinum 1-I...
3       one  16X20X1 (15.75 x 19.75)Accumulair Platinum 1-I...
4       get  16X20X1 (15.75 x 19.75)Accumulair Platinum 1-I...
...     ...                                                ...
1395    one  wall26 - Canvas Wall Art - Blue Oldtimer Farmi...
1396   back  wall26 - Canvas Wall Art - Blue Oldtimer Farmi...
1397  frame  wall26 - Canvas Wall Art - Blue Oldtimer Farmi...
1398  would  wall26 - Canvas Wall Art - Blue Oldtimer Farmi...
1399     br  wall26 - Canvas Wall Art - Blue Oldtimer Farmi...

[1400 rows x 2 columns]


In [8]:
final_df.sort_values(by='topic')

,topic,product_id
916,back,"PHINGEER Coat Rack Wall Mount, Heavy Duty Stai..."
466,back,FLOWEROOM 100% Blackout Velvet Curtains for Be...
296,back,Clothes Hanger Connector Hooks Hanger Extender...
1281,back,Vintage Apple Crate Farm Barnwood Primitive Cr...
131,back,"ArtToFrames 18x24 Inch White Picture Frame, Th..."
...,...,...
603,would,Haperlare Buffalo Tier Curtains for Kitchen Wi...
1038,would,Resinta 6 Pack Red Velvet Christmas Bow for Ho...
1118,would,Scholartree Rustic White 5x7 Picture Frame set...
558,would,"Goodbath Space Blackout Curtains, Outer Space ..."


In [11]:
final_df['topic'].value_counts()

topic
love       152
sheet      144
one         87
br          87
keep        86
easy        86
work        86
perfect     86
soft        66
color       66
bed         66
would       50
frame       50
back        50
great       41
price       41
quality     41
good        41
get         37
like        37
Name: count, dtype: int64